In [1]:
import yfinance as yf
import pandas as pd
from datetime import date, timedelta

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
import time
from io import StringIO

from SqlAlquemyInsertMarketDataHandler import SqlAlquemyInsertMarketDataHandler
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
historic_data = pd.read_csv('market_data/stock_indices.csv', parse_dates=True, index_col=0)
historic_data.tail()

,United States,Japan,United Kingdom,Canada,France,Switzerland,Germany,Australia,Netherlands,Denmark,...,Turkiye,Philippines,Poland,Chile,Greece,Hungary,Czechia,Colombia,Argentina,Russia
2023-12-11,4622.44,32791.80,7544.9,20318.4,7551.53,11130.36,16794.43,7199.0,786.45,2178.44,...,7728.4,6228.29,2313.32,5884.78,1275.50,4790.50,1408.27,3614.24,976823.12,3026.17
2023-12-12,4643.70,32843.70,7542.8,20233.8,7543.55,11151.22,16791.74,7235.3,786.13,2149.68,...,7748.9,6292.39,2293.66,5913.50,1283.12,4856.85,1406.03,3620.08,1010022.00,3019.89
2023-12-13,4707.09,32926.35,7548.4,20629.5,7531.22,11188.91,16766.05,7257.8,787.02,2180.53,...,7529.3,6255.74,2298.31,6020.41,1292.35,4844.99,1397.88,3675.84,1003484.00,3032.22
2023-12-14,4719.55,32686.25,7649.0,20778.8,7575.85,11209.95,16752.23,7377.9,789.91,2187.27,...,7813.4,6410.48,2312.56,6146.59,1295.37,5002.95,1388.53,3696.49,989696.00,3008.84
2023-12-15,4719.19,32970.55,7576.4,20529.2,7596.91,11191.89,16751.44,7442.7,793.13,2188.83,...,7993.9,6478.44,2320.45,6138.10,1290.79,4988.28,1379.69,3609.84,925658.00,3033.89


In [21]:
start_date = historic_data.last_valid_index() + timedelta(days = 1)
start_date = str(start_date.date())
start_date

'2023-12-16'

In [23]:
end_date = date.today() - timedelta(days = 1)
end_date = str(end_date)
end_date

'2024-01-02'

In [24]:
yahoo_references = {
    'United States': '^GSPC',  # S&P 500
    'Japan': '^N225',  # Nikkei 225
    'United Kingdom': '^FTSE',  # FTSE 100
    'Canada': '^GSPTSE',  # S&P/TSX Composite index
    'France': '^FCHI',  # CAC 40
    'Switzerland': '^SSMI',  # SMI PR
    'Germany': '^GDAXI',  # DAX PERFORMANCE-INDEX
    'Australia': '^AXJO',  # S&P/ASX 200
    'Netherlands': '^AEX',  # AEX-Index
    'Denmark': '',
    'Sweden': '^OMX',  # OMX Stockholm 30 Index
    'Spain': '^IBEX',  # IBEX 35
    'Hong Kong': '^HSI',  # HANG SENG INDEX
    'Italy': 'FTSEMIB.MI',  # FTSE MIB Index
    'Singapore': '^STI',  # STI Index
    'Finland': '^OMXH25',  # OMX Helsinki 25
    'Belgium': '^BFX',  # BEL 20
    'Norway': '',
    'Israel': '^TA125.TA',  # TA-125
    'Ireland': '^ISEQ',  # ISEQ All Share
    'New Zealand': '^NZ50',  # S&P/NZX 50 INDEX GROSS
    'Austria': '^ATX',  # Austrian Traded Index
    'Portugal': 'PSI20.LS',  # PSI 20
    'Eurozona': '^STOXX50E',  # Euro Stoxx 50

    'China': '000001.SS',  # SSE Composite Index
    'Taiwan': '^TWII',  # TSEC weighted index
    'India': '^BSESN',  # S&P BSE SENSEX
    'Korea': '^KS11',  # KOSPI Composite Index
    'Brazil': '^BVSP',  # IBOVESPA
    'Saudi Arabia': '^TASI.SR',  # Tadawul All Shares Index
    'South Africa': '',
    'Mexico': '^MXX',  # IPC MEXICO
    'Thailand': '^SET.BK',  # SET Index
    'Indonesia': '^JKSE',  # IDX COMPOSITE
    'Malaysia': '^KLSE',  # FTSE Bursa Malaysia KLCI
    'Turkiye': 'XU100.IS',  # BIST 100
    'Philippines': 'PSEI.PS',  # PSEi INDEX
    'Poland': '',
    'Chile': '',
    'Greece': 'GD.AT',  # COMPOSITE INDEX
    'Hungary': '',
    'Czechia': '',
    'Colombia': '',
    'Argentina': '^MERV',  # MERVAL
    'Russia': 'IMOEX.ME'  # MOEX Russia Index
}

In [25]:
df = pd.DataFrame(index=pd.date_range(start_date, end_date, freq='B'),
                  columns=yahoo_references)

for country in yahoo_references:
    index_ticker = yahoo_references[country]
    if index_ticker != '':
        index_data = yf.download(index_ticker, start_date)
        df[country] = index_data['Adj Close']

df

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

,United States,Japan,United Kingdom,Canada,France,Switzerland,Germany,Australia,Netherlands,Denmark,...,Turkiye,Philippines,Poland,Chile,Greece,Hungary,Czechia,Colombia,Argentina,Russia
2023-12-18,4740.560059,32758.980469,7614.500000,20622.699219,7568.859863,11155.799805,16650.550781,7426.399902,789.150024,NaN,...,7790.399902,6475.500000,NaN,NaN,1287.219971,NaN,NaN,NaN,921877.0,3076.260010
2023-12-19,4768.370117,33219.390625,7638.000000,20839.599609,7574.669922,11146.429688,16744.410156,7489.100098,792.479980,NaN,...,7725.899902,6521.270020,NaN,NaN,1298.479980,NaN,NaN,NaN,914461.0,3083.639893
2023-12-20,4698.350098,33675.941406,7715.700195,20600.800781,7583.430176,11144.879883,16733.050781,7537.899902,793.690002,NaN,...,7662.899902,6520.750000,NaN,NaN,1301.810059,NaN,NaN,NaN,919911.0,3104.659912
2023-12-21,4746.750000,33140.468750,7694.700195,20765.699219,7571.399902,11132.599609,16687.419922,7504.100098,789.880005,NaN,...,7765.000000,6469.229980,NaN,NaN,1298.560059,NaN,NaN,NaN,942904.0,3073.570068
2023-12-22,4754.629883,33169.050781,7697.500000,20881.199219,7568.819824,11153.089844,16706.179688,7501.600098,785.450012,NaN,...,7557.600098,6501.000000,NaN,NaN,1290.890015,NaN,NaN,NaN,948474.0,3092.580078
2023-12-25,NaN,33254.031250,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-12-26,4774.750000,33305.851562,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7299.299805,NaN,NaN,NaN,NaN,NaN,NaN,NaN,913125.0,3094.719971
2023-12-27,4781.580078,33681.238281,7725.000000,21015.900391,7571.819824,11113.980469,16742.070312,7561.200195,786.840027,NaN,...,7260.399902,6462.799805,NaN,NaN,1293.189941,NaN,NaN,NaN,872972.0,3097.489990
2023-12-28,4783.350098,33539.621094,7722.700195,20929.400391,7535.160156,11057.549805,16701.550781,7614.299805,787.039978,NaN,...,7396.299805,6519.109863,NaN,NaN,1285.650024,NaN,NaN,NaN,904067.0,3101.989990
2023-12-29,4769.830078,33464.171875,7733.200195,20958.400391,7543.180176,11137.790039,16751.640625,7590.799805,786.820007,NaN,...,7470.200195,6450.040039,NaN,NaN,1293.140015,NaN,NaN,NaN,NaN,3099.110107


In [26]:
investing_pages = {
    'Denmark': 'omx-copenhagen-20-historical-data',
    'Norway': 'oslo-obx-historical-data',
    'South Africa': 'ftse-jse-top-40-historical-data',
    'Poland': 'wig-20-historical-data',
    'Chile': 'ipsa-historical-data',
    'Hungary': 'ftse-hungary-historical-data',
    'Czechia': 'px-historical-data',
    'Colombia': 'ftse-colombia-historical-data'
}

In [27]:
def retrieve_prices_from_investing(path):
    service = Service(executable_path='../chromedriver.exe')
    options = webdriver.ChromeOptions()
    driver = webdriver.Chrome(service=service, options=options)

    base_url = 'https://www.investing.com/indices/'
    driver.get(f'{base_url}{path}')
    time.sleep(2)

    botton_cookies = driver.find_element(
        By.XPATH, '//*[@id="onetrust-accept-btn-handler"]'
    )
    botton_cookies.click()
    time.sleep(2)

    table = driver.find_element(
        By.XPATH, '//*[@id="__next"]'
    )
    dfs = pd.read_html(StringIO(table.get_attribute('innerHTML')))
    if path == 'oslo-obx-historical-data' or path == 'ftse-jse-top-40-historical-data':
        df_raw = dfs[1]
    else:
        df_raw = dfs[0]
    driver.close()

    df_raw.index = pd.to_datetime(df_raw['Date'])
    df_raw = df_raw.sort_index()
    return df_raw['Price']

In [28]:
for country in investing_pages:
    path = investing_pages[country]
    df_scrapped = retrieve_prices_from_investing(path)
    print(country, len(df_scrapped))
    df[country] = df_scrapped

Denmark 20
Norway 20
South Africa 19
Poland 20
Chile 20
Hungary 21
Czechia 19
Colombia 21


In [29]:
df

,United States,Japan,United Kingdom,Canada,France,Switzerland,Germany,Australia,Netherlands,Denmark,...,Turkiye,Philippines,Poland,Chile,Greece,Hungary,Czechia,Colombia,Argentina,Russia
2023-12-18,4740.560059,32758.980469,7614.500000,20622.699219,7568.859863,11155.799805,16650.550781,7426.399902,789.150024,2214.57,...,7790.399902,6475.500000,2334.50,6051.52,1287.219971,5012.29,1381.84,3603.73,921877.0,3076.260010
2023-12-19,4768.370117,33219.390625,7638.000000,20839.599609,7574.669922,11146.429688,16744.410156,7489.100098,792.479980,2228.08,...,7725.899902,6521.270020,2354.57,6060.44,1298.479980,5051.85,1382.38,3618.69,914461.0,3083.639893
2023-12-20,4698.350098,33675.941406,7715.700195,20600.800781,7583.430176,11144.879883,16733.050781,7537.899902,793.690002,2259.00,...,7662.899902,6520.750000,2346.52,6073.86,1301.810059,5067.33,1395.51,3595.24,919911.0,3104.659912
2023-12-21,4746.750000,33140.468750,7694.700195,20765.699219,7571.399902,11132.599609,16687.419922,7504.100098,789.880005,2272.59,...,7765.000000,6469.229980,2348.67,6107.78,1298.560059,5015.09,1395.80,3635.12,942904.0,3073.570068
2023-12-22,4754.629883,33169.050781,7697.500000,20881.199219,7568.819824,11153.089844,16706.179688,7501.600098,785.450012,2267.75,...,7557.600098,6501.000000,2346.70,6153.66,1290.890015,4995.13,1399.12,3628.52,948474.0,3092.580078
2023-12-25,NaN,33254.031250,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,4995.13,NaN,3628.52,NaN,NaN
2023-12-26,4774.750000,33305.851562,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7299.299805,NaN,NaN,6218.05,NaN,4995.13,NaN,3656.62,913125.0,3094.719971
2023-12-27,4781.580078,33681.238281,7725.000000,21015.900391,7571.819824,11113.980469,16742.070312,7561.200195,786.840027,2266.57,...,7260.399902,6462.799805,2371.28,6201.66,1293.189941,4991.95,1406.52,3669.47,872972.0,3097.489990
2023-12-28,4783.350098,33539.621094,7722.700195,20929.400391,7535.160156,11057.549805,16701.550781,7614.299805,787.039978,2283.04,...,7396.299805,6519.109863,2356.32,6249.47,1285.650024,5033.25,1409.99,3702.33,904067.0,3101.989990
2023-12-29,4769.830078,33464.171875,7733.200195,20958.400391,7543.180176,11137.790039,16751.640625,7590.799805,786.820007,2283.53,...,7470.200195,6450.040039,2342.99,6197.77,1293.140015,5016.69,1414.02,3702.33,NaN,3099.110107


In [34]:
df = df.ffill()
df = df.bfill()
df = df.round(2)
df

,United States,Japan,United Kingdom,Canada,France,Switzerland,Germany,Australia,Netherlands,Denmark,...,Turkiye,Philippines,Poland,Chile,Greece,Hungary,Czechia,Colombia,Argentina,Russia
2023-12-18,4740.56,32758.98,7614.5,20622.7,7568.86,11155.80,16650.55,7426.4,789.15,2214.57,...,7790.4,6475.50,2334.50,6051.52,1287.22,5012.29,1381.84,3603.73,921877.0,3076.26
2023-12-19,4768.37,33219.39,7638.0,20839.6,7574.67,11146.43,16744.41,7489.1,792.48,2228.08,...,7725.9,6521.27,2354.57,6060.44,1298.48,5051.85,1382.38,3618.69,914461.0,3083.64
2023-12-20,4698.35,33675.94,7715.7,20600.8,7583.43,11144.88,16733.05,7537.9,793.69,2259.00,...,7662.9,6520.75,2346.52,6073.86,1301.81,5067.33,1395.51,3595.24,919911.0,3104.66
2023-12-21,4746.75,33140.47,7694.7,20765.7,7571.40,11132.60,16687.42,7504.1,789.88,2272.59,...,7765.0,6469.23,2348.67,6107.78,1298.56,5015.09,1395.80,3635.12,942904.0,3073.57
2023-12-22,4754.63,33169.05,7697.5,20881.2,7568.82,11153.09,16706.18,7501.6,785.45,2267.75,...,7557.6,6501.00,2346.70,6153.66,1290.89,4995.13,1399.12,3628.52,948474.0,3092.58
2023-12-25,4754.63,33254.03,7697.5,20881.2,7568.82,11153.09,16706.18,7501.6,785.45,2267.75,...,7557.6,6501.00,2346.70,6153.66,1290.89,4995.13,1399.12,3628.52,948474.0,3092.58
2023-12-26,4774.75,33305.85,7697.5,20881.2,7568.82,11153.09,16706.18,7501.6,785.45,2267.75,...,7299.3,6501.00,2346.70,6218.05,1290.89,4995.13,1399.12,3656.62,913125.0,3094.72
2023-12-27,4781.58,33681.24,7725.0,21015.9,7571.82,11113.98,16742.07,7561.2,786.84,2266.57,...,7260.4,6462.80,2371.28,6201.66,1293.19,4991.95,1406.52,3669.47,872972.0,3097.49
2023-12-28,4783.35,33539.62,7722.7,20929.4,7535.16,11057.55,16701.55,7614.3,787.04,2283.04,...,7396.3,6519.11,2356.32,6249.47,1285.65,5033.25,1409.99,3702.33,904067.0,3101.99
2023-12-29,4769.83,33464.17,7733.2,20958.4,7543.18,11137.79,16751.64,7590.8,786.82,2283.53,...,7470.2,6450.04,2342.99,6197.77,1293.14,5016.69,1414.02,3702.33,904067.0,3099.11


In [36]:
df_new = pd.concat([historic_data])
df_new.to_csv('market_data/stock_indices.csv')

In [37]:
def save_to_db(historic_data, category):
    sql_handler = SqlAlquemyInsertMarketDataHandler()

    for country in historic_data.columns:     
            print(country, end='')   
            values = historic_data[country].dropna()
            symbol_code = sql_handler.get_symbol_code(category, country)
            if (len(values) > 0) and symbol_code != '':
                sql_handler.save_to_db(symbol_code, values)
            else:
                print(' - skipped')

In [38]:
save_to_db(df, 'Stock Index')

United States.Inserted 12 records for ^GSPC
Japan.Inserted 12 records for ^N225
United Kingdom.Inserted 12 records for ^FTSE
Canada.Inserted 12 records for ^GSPTSE
France.Inserted 12 records for ^FCHI
Switzerland.Inserted 12 records for ^SSMI
Germany.Inserted 12 records for ^GDAXI
Australia.Inserted 12 records for ^AXJO
Netherlands.Inserted 12 records for ^AEX
Denmark.Inserted 12 records for OMXC20
Sweden.Inserted 12 records for ^OMX
Spain.Inserted 12 records for ^IBEX
Hong Kong.Inserted 12 records for ^HSI
Italy.Inserted 12 records for FTSEMIB.MI
Singapore.Inserted 12 records for ^STI
Finland.Inserted 12 records for ^OMXH25
Belgium.Inserted 12 records for ^BFX
Norway.Inserted 12 records for OBX
Israel.Inserted 12 records for ^TA125.TA
Ireland.Inserted 12 records for ^ISEQ
New Zealand.Inserted 12 records for ^NZ50
Austria.Inserted 12 records for ^ATX
Portugal.Inserted 12 records for PSI20.LS
Eurozona.Inserted 12 records for ^STOXX50E
China.Inserted 12 records for 000001.SS
Taiwan.Inser